In [1]:
import numpy as np
from collections import defaultdict as dd
import pandas as pd
import gc
from sklearn.linear_model import LinearRegression as LR

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib notebook
pd.set_option('display.max_rows', 500)

from valkyrie.avalon.feature_analyzer import FeatureMgr
from valkyrie.securities import stocks_good_dvd, parent
from valkyrie.quants import lm_fit

/tmp/ipykernel_591279/3906435911.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Load Data

In [2]:
#feature_mgr = FeatureMgr('20210101', '20210630', ['TWO PRC'] , 'latest')#
feature_mgr = FeatureMgr('20210101', '20210630', stocks_good_dvd() , 'eve_recording3')
rets = [1 , 2 , 5, 10, 12, 15]
ret_ns = [int(n * 2 * 60 * 6.5) for n in rets]

ret_col, s2_col = 'cywa_mkt_xh', 'cywa_mkt_s2h'
df = feature_mgr.calc_rets(ret_col, ret_ns, s2_col)
gc.collect()
df['parent'] = df['ticker'].apply(parent)

# Cluster fitting

In [4]:
df_cluster = df.copy()

In [11]:
r2s = dd(lambda : {})
res = {}
yhz = 780
for hz in ['long', 'vlong', 'vvlong', 'vvvlong']:
    print(f'========= {hz} ========')
    
    #ycols = [f'cywa_mkt_xh_ret_{yhz}_n' for yhz in ret_ns]
    #wcols = [f'total_wgt_cywa_mkt_xh_ret_{yhz}_n' for yhz in ret_ns]
    
    #df_cluster[f'feature_{hz}'] = df_cluster.eval(f' : ')        
    df_cluster[f'feature_{hz}'] = df_cluster.eval(f'cluster_{hz}_avg_xh - (cywa_{hz}_xh - cywa_mkt_xh)')

    ycol = f'cywa_mkt_xh_ret_{yhz}_n'
    wcol = f'total_wgt_cywa_mkt_xh_ret_{yhz}_n'
    df_cluster[wcol] = df_cluster.eval(f'cywa_mkt_s2h + cywa_{hz}_s2h + cluster_{hz}_avg_s2 + (1.0 / wgt_cywa_mkt_xh_ret_{yhz}_n)')
    df_cluster[wcol] = df_cluster.eval(f'1.0/{wcol}')

    feature= f'feature_{hz}'
    df_z = df_cluster.query(f'abs({feature}) >= 1.0 * 1.0e-4')

    res[hz] = lm_fit(df_z, [f'feature_{hz}'], [ycol], [wcol])
    display(res[hz])
                
#r2s = pd.DataFrame(r2s)

========= long ========


,cywa_mkt_xh_ret_780_n
r2,0.03196
feature_long,-0.399869
model,Linefit_False_q=0.03


========= vlong ========


,cywa_mkt_xh_ret_780_n
r2,0.027766
feature_vlong,-0.33448
model,Linefit_False_q=0.03


========= vvlong ========


,cywa_mkt_xh_ret_780_n
r2,0.026001
feature_vvlong,-0.312218
model,Linefit_False_q=0.03


========= vvvlong ========


,cywa_mkt_xh_ret_780_n
r2,0.026
feature_vvvlong,-0.31221
model,Linefit_False_q=0.03


In [ ]:
df_cluster[[c for c in df_cluster if 'feature_' in c]].describe()

# Trade Imbalance Fitting

In [ ]:
df_ti = df.copy()

In [ ]:
df_ti.columns

In [ ]:
for net in ['vshort', 'short', 'mid', 'long']:
    df_ti[f'net_{net}'] = df_ti.eval(f'buytrade_{net} - selltrade_{net}')
    df_ti[f'cluster_net_{net}'] = df_ti.eval(f'cluster_buy_{net} - cluster_sell_{net}')
    for gross in ['long', 'vlong']:
        df_ti[f'gross_{gross}'] = df_ti.eval(f'buytrade_{gross} + selltrade_{gross} + midtrade_{gross}')
        df_ti[f'feature_{net}_{gross}'] = df_ti.eval(f'(net_{net} / (gross_{gross})) * cywa_mid_xh')        
        df_ti[f'feature_{net}_{gross}'] = df_ti[f'feature_{net}_{gross}'].fillna(0.0)
        
        df_ti[f'cluster_gross_{gross}'] = df_ti.eval(f'cluster_buy_{gross} + cluster_sell_{gross} + cluster_mid_{gross}')
        df_ti[f'feature_cluster_{net}_{gross}'] = df_ti.eval(f'(cluster_net_{net} / (cluster_gross_{gross})) * cywa_mid_xh')        
        df_ti[f'feature_cluster_{net}_{gross}'] = df_ti[f'feature_cluster_{net}_{gross}'].fillna(0.0)

In [ ]:
df_ti.query('date >= "20210215"')[['ticker'] + [c for c in df_ti if 'net_' in c or 'gross_' in c or 'feature_' in c]].describe()

In [ ]:
df_ti.query('date >= "20210215"')[['ticker'] + [c for c in df_ti if 'feature_' in c]].describe()

In [ ]:
df_ti_top = df_ti.query('date >= "20210215"').copy()

In [ ]:
features = [f for f in df_ti if 'feature_' in f and 'vshort' in f]

r2s = dd(lambda : {})
for feature in features:    
    print(feature)
    for yhz in ret_ns:
        #print(f'====== {yhz} =========')
        ycol = f'cywa_mkt_xh_ret_{yhz}_n'
        wcol = f'wgt_cywa_mkt_xh_ret_{yhz}_n'
        df_z = df_ti_top.query(f'abs({feature}) > 1e-4')[[feature, ycol, wcol]].copy()        
        r2, n, coeffs = lm_fit(df_z, [feature], [ycol], [wcol])
        r2s[yhz][feature] = (r2, n)
            
r2s = pd.DataFrame(r2s)

In [ ]:
r2s